In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers

import os

In [ ]:
PATH = "/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge"

data = pd.read_csv(os.path.join(PATH, "icml_face_data.csv"))

emotions = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}

In [ ]:
# Function to parse data into right format
# Output: Image in right shaped and normalized + labels
def parse_data(data):
    image_array = np.zeros(shape=(len(data), 48, 48, 1))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, ' pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48, 1))
        image_array[i] = image
        
    return image_array, image_label

# Splitting the data into train, validation and testing set thanks to Usage column
train_imgs, train_lbls = parse_data(data[data[" Usage"] == "Training"])
val_imgs, val_lbls = parse_data(data[data[" Usage"] == "PrivateTest"])
test_imgs, test_lbls = parse_data(data[data[" Usage"] == "PublicTest"])

In [ ]:
# Building a CNN model based on LeNet architecture 
# See: https://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/#:~:text=The%20LeNet%20architecture%20is%20an,enough%20to%20provide%20interesting%20results.
model = keras.Sequential()

model.add(layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(48, 48,1)))
model.add(layers.AveragePooling2D())

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(layers.AveragePooling2D())

model.add(layers.Flatten())

model.add(layers.Dense(units=120, activation='relu'))

model.add(layers.Dense(units=84, activation='relu'))

model.add(layers.Dense(units=7, activation = 'softmax'))

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])

model.summary()

In [ ]:
# Training the model, and validating
model.fit(train_imgs, train_lbls, 
          epochs=30, batch_size=32, 
          validation_data=(val_imgs, val_lbls), verbose=1)

In [ ]:
# inp = keras.Input((48, 48, 1))
# outputs = keras.applications.ResNet50(weights=None, input_shape=(48, 48, 1), classes=7)(inp)
# model = keras.Model(inp, outputs)
# model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
# model.summary()

In [ ]:
# model.fit(train_imgs, train_lbls, 
#           epochs=50, batch_size=32, 
#           validation_data=(val_imgs, val_lbls), verbose=1)

In [ ]:
# Plotting confusion matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

test_preds = model.predict(test_imgs).argmax(axis=1)

conf_mat = confusion_matrix(test_lbls, test_preds)
fig, ax = plot_confusion_matrix(conf_mat=conf_mat,
                                show_normed=True,
                                show_absolute=False,
                                class_names=emotions.values(),
                                figsize=(8, 8))